In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import gc
import collections
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/datapart2/y_p2.npy
/kaggle/input/datapart2/encoding_p2.npy
/kaggle/input/datapart5/__notebook_source__.ipynb
/kaggle/input/datapart5/encoding_p5.npy
/kaggle/input/datapart5/y_p5.npy
/kaggle/input/datapart3/y_p3.npy
/kaggle/input/datapart3/encoding_p3.npy
/kaggle/input/datapart4/encoding_p4.npy
/kaggle/input/datapart4/__notebook_source__.ipynb
/kaggle/input/datapart4/y_p4.npy
/kaggle/input/test-encoding/encoding_test.npy
/kaggle/input/amazon-ml-challenge-2021-hackerearth/sample_submission.csv
/kaggle/input/amazon-ml-challenge-2021-hackerearth/train.csv
/kaggle/input/amazon-ml-challenge-2021-hackerearth/test.csv
/kaggle/input/datapart1/y_p1.npy
/kaggle/input/datapart1/encoding_p1.npy


In [2]:
with open('../input/datapart1/y_p1.npy', 'rb') as f:
    y1 = np.load(f)
with open('../input/datapart2/y_p2.npy', 'rb') as f:
    y1 = np.concatenate((y1,np.load(f)))
with open('../input/datapart3/y_p3.npy', 'rb') as f:
    y1 = np.concatenate((y1,np.load(f)))
with open('../input/datapart4/y_p4.npy', 'rb') as f:
    y1 = np.concatenate((y1,np.load(f)))

In [3]:
label = {}
for i,j in enumerate(np.unique(y1)):
    label[j]=i

In [4]:
classes = len(np.unique(y1))

In [5]:
classes

1462

In [6]:
from keras.models import Sequential
from keras.layers import Dense,SpatialDropout1D,MaxPool1D,BatchNormalization, Dropout
import tensorflow as tf

my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, min_delta=0.0001),
    tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=5, verbose=1,
    mode='auto', min_delta=0.0001)
]

model = Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(1024,)))
#model.add(tf.keras.layers.LSTM(512))
model.add(Dense(1024*2, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(1536, activation='relu'))
# model.add(Dropout(0.3))
#model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(2048*2)))
model.add(Dense(classes,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
del y1

In [8]:
with open('../input/datapart1/encoding_p1.npy', 'rb') as f:
    X = np.load(f)
with open('../input/datapart2/encoding_p2.npy', 'rb') as f:
    X = np.concatenate((X,np.load(f)))
    
with open('../input/datapart1/y_p1.npy', 'rb') as f:
    y1 = np.load(f)
with open('../input/datapart2/y_p2.npy', 'rb') as f:
    y1 = np.concatenate((y1,np.load(f)))

In [9]:
gc.collect()

344

In [10]:
y1 = np.array([label[i] for i in y1])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.33, random_state=42)

In [12]:
del X,y1
gc.collect()

66

In [13]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1])
model.fit(X_train,y_train,epochs=500,validation_data=(X_test, y_test),batch_size = 128,callbacks=my_callbacks)

Epoch 1/500
5074/5074 [==============================] - 23s 4ms/step - loss: 2.2597 - accuracy: 0.5430 - val_loss: 1.3550 - val_accuracy: 0.6771
Epoch 2/500
5074/5074 [==============================] - 20s 4ms/step - loss: 1.2046 - accuracy: 0.7034 - val_loss: 1.2625 - val_accuracy: 0.6982
Epoch 3/500
5074/5074 [==============================] - 20s 4ms/step - loss: 1.0055 - accuracy: 0.7432 - val_loss: 1.2313 - val_accuracy: 0.7082
Epoch 4/500
5074/5074 [==============================] - 19s 4ms/step - loss: 0.8522 - accuracy: 0.7763 - val_loss: 1.2227 - val_accuracy: 0.7143
Epoch 5/500
5074/5074 [==============================] - 20s 4ms/step - loss: 0.7204 - accuracy: 0.8054 - val_loss: 1.2493 - val_accuracy: 0.7143
Epoch 6/500
5074/5074 [==============================] - 20s 4ms/step - loss: 0.6127 - accuracy: 0.8306 - val_loss: 1.2882 - val_accuracy: 0.7163
Epoch 7/500
5074/5074 [==============================] - 19s 4ms/step - loss: 0.5226 - accuracy: 0.8538 - val_loss: 1.3374 -

In [14]:
model.save('./Model1_1.h5',save_format='h5')

In [15]:
# X_test = X_test.reshape(X_test.shape[0],X_test.shape[1])
y_pred = np.argmax(model.predict(X_test),axis=1)

In [16]:
accuracy_score(y_test, y_pred)

0.7285052879039393

In [17]:
del X_train, X_test, y_train, y_test
gc.collect()

1479

In [18]:
with open('../input/datapart3/encoding_p3.npy', 'rb') as f:
    X = np.load(f)
with open('../input/datapart4/encoding_p4.npy', 'rb') as f:
    X = np.concatenate((X,np.load(f)))
with open('../input/datapart3/y_p3.npy', 'rb') as f:
    y1 = np.load(f)
with open('../input/datapart4/y_p4.npy', 'rb') as f:
    y1 = np.concatenate((y1,np.load(f)))

In [19]:
y1 = np.array([label[i] for i in y1])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.33, random_state=42)

In [21]:
del X,y1
gc.collect()

120

In [22]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, min_delta=0.0001),
    tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=5, verbose=1,
    mode='auto', min_delta=0.0001)
]

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1])
model.fit(X_train,y_train,epochs=500,validation_data=(X_test, y_test),batch_size = 128,callbacks=my_callbacks)

Epoch 1/500
5074/5074 [==============================] - 21s 4ms/step - loss: 1.5665 - accuracy: 0.7053 - val_loss: 1.5292 - val_accuracy: 0.7050
Epoch 2/500
5074/5074 [==============================] - 20s 4ms/step - loss: 1.4921 - accuracy: 0.7060 - val_loss: 1.4828 - val_accuracy: 0.7042
Epoch 3/500
5074/5074 [==============================] - 20s 4ms/step - loss: 1.4478 - accuracy: 0.7065 - val_loss: 1.4537 - val_accuracy: 0.7039
Epoch 4/500
5074/5074 [==============================] - 20s 4ms/step - loss: 1.4168 - accuracy: 0.7075 - val_loss: 1.4336 - val_accuracy: 0.7040
Epoch 5/500
5074/5074 [==============================] - 19s 4ms/step - loss: 1.3932 - accuracy: 0.7087 - val_loss: 1.4187 - val_accuracy: 0.7042
Epoch 6/500
5074/5074 [==============================] - 20s 4ms/step - loss: 1.3737 - accuracy: 0.7104 - val_loss: 1.4068 - val_accuracy: 0.7048
Epoch 7/500
5074/5074 [==============================] - 20s 4ms/step - loss: 1.3568 - accuracy: 0.7119 - val_loss: 1.3968 -

In [23]:
model.save('./Model1_2.h5',save_format='h5')

In [24]:
# X_test = X_test.reshape(X_test.shape[0],X_test.shape[1])
# y_pred = np.argmax(model.predict(X_test),axis=1)

In [25]:
# accuracy_score(y_test, np.argmax(model.predict(X_test),axis=1))

In [26]:
del X_train, X_test, y_train, y_test
gc.collect()

1880

In [28]:
with open('../input/datapart5/encoding_p5.npy', 'rb') as f:
    X = np.load(f)
with open('../input/datapart5/y_p5.npy', 'rb') as f:
    y1 = np.load(f)

In [29]:
y1 = np.array([label[i] for i in y1])
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.33, random_state=42)
del X,y1
gc.collect()
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1])
model.fit(X_train,y_train,epochs=500,validation_data=(X_test, y_test),batch_size = 128,callbacks=my_callbacks)

Epoch 1/500
3304/3304 [==============================] - 14s 4ms/step - loss: 1.0372 - accuracy: 0.7653 - val_loss: 1.0234 - val_accuracy: 0.7665
Epoch 2/500
3304/3304 [==============================] - 13s 4ms/step - loss: 1.0369 - accuracy: 0.7653 - val_loss: 1.0232 - val_accuracy: 0.7665
Epoch 3/500
3304/3304 [==============================] - 13s 4ms/step - loss: 1.0366 - accuracy: 0.7653 - val_loss: 1.0231 - val_accuracy: 0.7666
Epoch 4/500
3304/3304 [==============================] - 13s 4ms/step - loss: 1.0364 - accuracy: 0.7653 - val_loss: 1.0229 - val_accuracy: 0.7666
Epoch 5/500
3304/3304 [==============================] - 13s 4ms/step - loss: 1.0361 - accuracy: 0.7653 - val_loss: 1.0227 - val_accuracy: 0.7666
Epoch 6/500
3304/3304 [==============================] - 13s 4ms/step - loss: 1.0359 - accuracy: 0.7653 - val_loss: 1.0225 - val_accuracy: 0.7666
Epoch 7/500
3304/3304 [==============================] - 13s 4ms/step - loss: 1.0356 - accuracy: 0.7654 - val_loss: 1.0224 -

In [ ]:
# X_test = X_test.reshape(X_test.shape[0],X_test.shape[1])
# y_pred = np.argmax(model.predict(X_test),axis=1)

In [30]:
model.save('./Model1_3.h5',save_format='h5')

In [ ]:
accuracy_score(y_test, np.argmax(model.predict(X_test),axis=1))

In [31]:
with open('../input/test-encoding/encoding_test.npy', 'rb') as f:
    test = np.load(f)

In [32]:
X_test = test.reshape(test.shape[0],test.shape[1])

In [33]:
y_pred = np.argmax(model.predict(X_test),axis=1)

In [34]:
t = pd.read_csv('../input/amazon-ml-challenge-2021-hackerearth/test.csv',escapechar='\\', quoting=3)

In [35]:
t.columns

Index(['PRODUCT_ID', 'TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'BRAND'], dtype='object')

In [36]:
invert = {v: k for k, v in label.items()}

In [37]:
predd = [invert[i] for i in y_pred]

In [38]:
sub = pd.DataFrame({'PRODUCT_ID':t['PRODUCT_ID'].values,'BROWSE_NODE_ID':predd})

In [39]:
sub.to_csv('sub4.csv',index=False)